# Pro Trader pure implementation: no broker algo, requires manual trading

In [1]:
from environment import BuyKnowledge, SellKnowledge, run_agent
from preprocessing import donchian_signals, test_wrapper, get_data
from datetime import datetime, timedelta

import yfinance as yf
from stable_baselines3 import PPO
import pandas as pd
import random

In [2]:
today = datetime.now().date()
start = str(today - timedelta(days = 571))
end = str(today)

In [35]:
# currently held stock: [cost basis, buy date]
balance = { 'HXL': [82.61, '2026-01-30'],
 'AIR': [105.03, '2026-01-30'],
 'LBRT': [24.405, '2026-01-30'],
 'HON': [226.83, '2026-01-30'],
 'ESE': [226.76, '2026-01-30'],
 'ATGE': [104.53, '2026-01-30'],
 'COR': [355.1, '2026-01-30'],
 'VICR': [165.055, '2026-01-30'],
 'DLX': [25.935, '2026-01-30'],
 'SEIC': [87.33, '2026-01-30'],
 'RES': [6.575, '2026-01-30'],
 'UCTT': [44.35, '2026-01-30'],
 'CAT': [656.52, '2026-01-30'],
 'AESI': [11.63, '2026-01-30'],
 'BKR': [55.61, '2026-01-30'],
 'CAH': [212.23, '2026-01-30'],
 'TPL': [342.85, '2026-01-30'],
 'WHD': [55.63, '2026-01-30'],
 'NVT': [112.955, '2026-01-30'],
 'ALKS': [34.2299, '2026-01-30'],
 'HII': [419.54, '2026-01-30'],
 'MAN': [35.42, '2026-01-30'],
 'LPG': [29.57, '2026-01-30'],
 'PTEN': [7.5, '2026-01-30'],
 'IBM': [304.39, '2026-01-30'],
 'PLXS': [202.8, '2026-01-30'],
 'HLX': [7.95, '2026-01-30'],
 'ETN': [351.15, '2026-01-30'],
 'MMC': [185.21, '2026-01-30'],
 'VSH': [19.89, '2026-01-30'],
 'VMI': [442.12, '2026-01-30'],
 'TALO': [11.93, '2026-01-30'],
 'TT': [417.41, '2026-01-30'],
 'CENX': [45.85, '2026-01-30'],
 'CASH': [89.055, '2026-01-30'],
 'LUV': [47.52, '2026-01-30'],
 'VLY': [12.48, '2026-01-30'],
 'SCHL': [34.895, '2026-01-30'],
 'JXN': [118.5, '2026-01-30'],
 'RCL': [319.29, '2026-01-30'],
 'TTEK': [37.65, '2026-01-30'],
 'CW': [657.51, '2026-01-30'],
 'VCTR': [69.88, '2026-01-30'],
 'HUBB': [485.37, '2026-01-30'],
 'T': [25.715, '2026-01-30'],
 'GOOG': [338.2496, '2026-01-30'],
 'MWA': [26.75, '2026-01-30'],
 'CMI': [572.305, '2026-01-30'],
 'RRX': [158.845, '2026-01-30'],
 'REG': [71.74, '2026-01-30'],
 'LHX': [342.6, '2026-01-30'],
 'FLO': [11.175, '2026-01-30'],
 'LECO': [263.74, '2026-01-30'],
 'KGS': [41.255, '2026-01-30'],
 'SMG': [64.04, '2026-01-30'],
 'EMR': [146.59, '2026-01-30'],
 'FIZZ': [33.6, '2026-01-30'],
 'PSX': [142.07, '2026-01-30']
}

In [4]:
buy_symbols = []
sideways_20 = {}
current_price = {}

symbol_list = [] 
for i in [5,4,6]:
    a = pd.read_excel(f'SP{i}00.xlsx')
    symbol_list += list(a.Symbol)

symbol_list = list(set(symbol_list))

In [5]:
for symbol in symbol_list: 
    df = donchian_signals(symbol)
    if len(df)>0:
        current_price[symbol] = df.iloc[-1].close
        if df.iloc[-1].buy_signal and not df.iloc[-2].buy_signal:
            buy_symbols.append(symbol)


1 Failed download:
['X']: YFPricesMissingError('possibly delisted; no price data found  (period=1mo) (Yahoo error = "No data found, symbol may be delisted")')

1 Failed download:
['BF.B']: YFPricesMissingError('possibly delisted; no price data found  (period=1mo)')

1 Failed download:
['BRK.B']: YFPricesMissingError('possibly delisted; no price data found  (period=1mo) (Yahoo error = "No data found, symbol may be delisted")')

1 Failed download:
['CHX']: YFPricesMissingError('possibly delisted; no price data found  (period=1mo) (Yahoo error = "No data found, symbol may be delisted")')

1 Failed download:
['AMED']: YFPricesMissingError('possibly delisted; no price data found  (period=1mo) (Yahoo error = "No data found, symbol may be delisted")')

1 Failed download:
['LNW']: YFPricesMissingError('possibly delisted; no price data found  (period=1mo) (Yahoo error = "No data found, symbol may be delisted")')

1 Failed download:
['LANC']: YFPricesMissingError('possibly delisted; no price da

In [6]:
sell_symbols = []
sell_returns = []
sell_data = pd.DataFrame()
for symbol in balance:
    last_buy_price = balance[symbol][0]

    stop_loss_data = get_data(symbol, start = balance[symbol][1], end = end)
    max_dd = current_price[symbol] / stop_loss_data.close.max() -1

    side = stop_loss_data.iloc[-120:]
    side['returns'] = side.close/ last_buy_price -1
    side['sideways'] = side.returns <0.1
    side['sideways_20'] = side.sideways.rolling(20).sum()
    
    # Stop Loss: sideways and dips
    if side.sideways_20.iloc[-1] == 20: 
        print(f'Sideways stop loss triggered for {symbol}')

    if max_dd < -0.1:
        print(f'Dip stop loss triggered for {symbol}')
        
    # Get data for other held symbols
    else:
        sell_return = current_price[symbol] / last_buy_price -1
        df = test_wrapper(symbol, start, end)
        sell_symbols.append(symbol)
        sell_returns.append(sell_return)
        sell_data = pd.concat([sell_data, df.iloc[-1,:-2].reset_index(drop=True)], axis = 1)
    
sell_data = sell_data.T
sell_data['sell_return'] = sell_returns
sell_data = sell_data.dropna()
sell_data.loc[len(sell_data)] = [0] * sell_data.shape[1] # add one row for terminal state

Sideways stop loss triggered for PLTR
Dip stop loss triggered for PLTR


In [7]:
# Initialise deployment environment and model
env = SellKnowledge(sell_data, train_mode = False)
model = PPO.load('sell_knowledge_agent')
actions = run_agent(env, model)

# Sell if P(sell) - P(hold) > 0.85
rank_sells = {}
for i in range(len(sell_symbols)):
    rank_sells[sell_symbols[i]] = actions[i]
rank_sells = pd.DataFrame(rank_sells).T.reset_index()
rank_sells['diff'] = rank_sells[0] - rank_sells[1]
rank_sells.to_csv('rank_sells.csv', index = False)
print(rank_sells)
rank_sells = rank_sells[rank_sells['diff'] > 0.85] 

# of steps: 3
  index    0         1      diff
0  META  1.0  0.000000  1.000000
1  AAPL  0.0  0.634514 -0.634514


In [18]:
# Get data for buy signals
buy_data = pd.DataFrame()
buy_symbols = random.sample(buy_symbols, len(buy_symbols))
failed = set()

for symbol in buy_symbols:
    try:
        df = test_wrapper(symbol, start, end)
        buy_data = pd.concat([buy_data, df.iloc[-1,:-2].reset_index(drop=True)], axis = 1)
    except:
        print(f'Not enough data for {symbol}')
        failed.add(symbol)
        
buy_data = buy_data.T
buy_data.loc[len(buy_data)] = [0] * buy_data.shape[1] # add one row for terminal state 

# Initialise deployment environment and model
env = BuyKnowledge(buy_data, train_mode = False)
model = PPO.load('buy_knowledge_agent')
actions = run_agent(env, model)

# Prioritize buy signals with higher probability of high returns
rank_buys = {}
i = -1
for a in actions:
    i+=1
    if i in failed:
        continue
    else:
        rank_buys[buy_symbols[i]] = a
        
rank_buys = pd.DataFrame(rank_buys).T.reset_index()
rank_buys['diff'] = rank_buys[0] - rank_buys[1]
rank_buys = rank_buys.sort_values('diff', ascending = False).reset_index(drop = True) 
rank_buys.to_csv('rank_buys.csv', index = False)
print(rank_buys)
rank_buys = rank_buys[rank_buys['diff'] > 0] # increase threshold for lower risk

Not enough data for AMTM
# of steps: 105
    index    0    1  diff
0     HON  0.0  0.0   0.0
1    VCTR  0.0  0.0   0.0
2     IBM  0.0  0.0   0.0
3     ESE  0.0  0.0   0.0
4     AOS  0.0  0.0   0.0
..    ...  ...  ...   ...
99   ATGE  0.0  1.0  -1.0
100  FFBC  0.0  1.0  -1.0
101    TT  0.0  1.0  -1.0
102   MWA  0.0  1.0  -1.0
103   REG  0.0  1.0  -1.0

[104 rows x 4 columns]


In [21]:
rank_buys

,index,0,1,diff


In [10]:
model = PPO.load('buy_knowledge_agent2')
actions = run_agent(env, model)

# Prioritize buy signals with higher probability of high returns
rank_buys2 = {}
i = -1
for a in actions:
    i+=1
    if i in failed:
        continue
    else:
        rank_buys2[buy_symbols[i]] = a
        
rank_buys2 = pd.DataFrame(rank_buys2).T.reset_index()
rank_buys2['diff'] = rank_buys2[0] - rank_buys2[1]
rank_buys2 = rank_buys2.sort_values('diff', ascending = False).reset_index(drop = True) 
rank_buys2.to_csv('rank_buys2.csv', index = False)
print(rank_buys2)
rank_buys2 = rank_buys2[rank_buys2['diff'] > 0] # increase threshold for lower risk

    index    0    1  diff
0     HXL  1.0  0.0   1.0
1     AIR  1.0  0.0   1.0
2    LBRT  1.0  0.0   1.0
3     HON  1.0  0.0   1.0
4     ESE  1.0  0.0   1.0
..    ...  ...  ...   ...
99    APA  0.0  1.0  -1.0
100   EOG  0.0  1.0  -1.0
101   FDX  0.0  1.0  -1.0
102  SCSC  0.0  1.0  -1.0
103  CALM  0.0  1.0  -1.0

[104 rows x 4 columns]


# for forward testing purposes only

In [ ]:
# update balance

today_bal = 0

for s in list(rank_buys2['index']):
    try:
        cost = yf.Ticker(s).info['regularMarketPrice']
    except:
        cost = yf.Ticker(s).info['regularMarketOpen']
    balance[s] = [cost, end]
    today_bal += cost

print(today_bal)

balance

In [ ]:
# calculate profit/loss/portfolio value
jan_30 = 9527.7195